In [1]:
import numpy as np 
import tensorflow as tf 
  
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.layers import LSTM 
  
from keras.optimizers import RMSprop 
  
from keras.callbacks import LambdaCallback 
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import ReduceLROnPlateau 
import random 
import sys

2024-04-10 10:47:14.168281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 10:47:14.301497: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 10:47:14.302532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 10:47:14.978856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# with open('data/article.txt', 'r') as file: 
#     text = file.read()     
# print(text) 

In [3]:
text = "Please remember that all group members must be present for the presentation. Should there be any issues with the schedule, please reach out to us."
# Storing all the unique characters present in the text 
vocabulary = sorted(list(set(text)))
print(len(vocabulary))
# vocabulary

25


In [4]:
# Creating dictionaries to map each character to an index 
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary)) 
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary)) 

In [5]:
max_length = 100
steps = 5
sentences = [] 
next_chars = []
for i in range(0, len(text) - max_length, steps): 
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
print(sentences)
print(next_chars)

['Please remember that all group members must be present for the presentation. Should there be any iss', 'e remember that all group members must be present for the presentation. Should there be any issues w', 'ember that all group members must be present for the presentation. Should there be any issues with t', ' that all group members must be present for the presentation. Should there be any issues with the sc', ' all group members must be present for the presentation. Should there be any issues with the schedul', 'group members must be present for the presentation. Should there be any issues with the schedule, pl', ' members must be present for the presentation. Should there be any issues with the schedule, please ', 'ers must be present for the presentation. Should there be any issues with the schedule, please reach', 'ust be present for the presentation. Should there be any issues with the schedule, please reach out ', 'e present for the presentation. Should there be any issues wit

In [6]:
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype=bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = bool)

In [7]:
for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

## Building the LSTM ##

In [11]:
model = Sequential() 
model.add(LSTM(128, input_shape =(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))
optimizer = RMSprop(learning_rate = 0.01) 
model.compile(loss ='categorical_crossentropy', optimizer = optimizer) 

In [12]:
# sample an index from a probability array 
def sample_index(preds, temperature = 1.0): 
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)

In [13]:
# generate text after each epoch
def on_epoch_end(epoch, logs): 
    print() 
    print('----- Generating text after Epoch: % d' % epoch) 
  
    start_index = random.randint(0, len(text) - max_length - 1) 
    for diversity in [0.2, 0.5, 1.0, 1.2]: 
        print('----- diversity:', diversity) 
  
        generated = '' 
        sentence = text[start_index: start_index + max_length] 
        generated += sentence 
        print('----- Generating with seed: "' + sentence + '"') 
        sys.stdout.write(generated) 
  
        for i in range(400): 
            x_pred = np.zeros((1, max_length, len(vocabulary))) 
            for t, char in enumerate(sentence): 
                x_pred[0, t, char_to_indices[char]] = 1.
  
            preds = model.predict(x_pred, verbose = 0)[0] 
            next_index = sample_index(preds, diversity) 
            next_char = indices_to_char[next_index] 
  
            generated += next_char 
            sentence = sentence[1:] + next_char 
  
            sys.stdout.write(next_char) 
            sys.stdout.flush() 
        print() 
print_callback = LambdaCallback(on_epoch_end = on_epoch_end) 

In [14]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor ='loss', 
                             verbose = 1, save_best_only = True, 
                             mode ='min') 

In [15]:
# Defining a helper function to reduce the learning rate each time 
# the learning plateaus 
reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2, 
                              patience = 1, min_lr = 0.001) 
callbacks = [print_callback, checkpoint, reduce_alpha] 

In [17]:
# Training the LSTM model 
model.fit(X, y, batch_size = 128, epochs = 10, callbacks = callbacks) 

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 1.1197
----- Generating text after Epoch:  0
----- diversity: 0.2
----- Generating with seed: "p members must be present for the presentation. Should there be any issues with the schedule, please"
p members must be present for the presentation. Should there be any issues with the schedule, pleaseeer  rrrrr r tttutt.tththhhhhhhhhhheeeeeeeeeeeeerr r   rrruttttttt..h.hhhhhhhhhheeeeeeeeeee  r rrrrrr ru.ttt.ttttthhhhhhhhhhheeeeeeeeeeerre  rrr rrttttttttt.thhhhhhhhhhhhheeeeeeeeererr rrrrr  tutt.uttt.hhhhhhhhhhhhheeeeeeeeee rrr  rrrrrr ttttt.t..h.hhhhhhhhheeheeeeeeeeerr rrrrr rrruutttttt.tth.hhhhhhhhhheeeeeeeeeeerrrr rr  rt rttttttth.hhhhhhhhhhhheeeeeeeee eer  rrrrr r rttrttt..tthhhhhhhhhhhheeee
----- diversity: 0.5
----- Generating with seed: "p members must be present for the presentation. Should there be any issues with the schedule, please"
p members must be present for the presentation. Should there be any issues with the 

mber that all group members must be present for the presentation. Should there be any issues with the.f.thrrieeie rrieyru iPti.uu.uueh.iltu.uhihuwrh .ySutueeesii.fu het ,umre.trrue. iy.ihu ueut.tr. riaiutiu.i.ti iiyrfheheh heeeu.uewbuu.re h.,yrtrueiuiut.e.i.tunh u .hiueih.tr.hew.iee  e.et hh.irtiueShih.tr.utiuuuuhiihS er.t ei. ue.ueyeg iuttei..httrhharriuhphbeuee g ruPhui l. uhht.mmhtef .hrre.eiti ..ehe  ihgth.,tiiri hht etthihhieyu rehrh. rthtfrhr.h.r h...iuyeih heuhuurhuuehh.hiueh..h t.rheuuh.

Epoch 3: loss did not improve from 1.11965
1/1 [==============================] - 47s 47s/step - loss: 1.7188 - lr: 0.0010
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 1.2215
----- Generating text after Epoch:  3
----- diversity: 0.2
----- Generating with seed: "t be present for the presentation. Should there be any issues with the schedule, please reach out to"
t be present for the presentation. Should there be any issues with the schedule, please reach out tott...h.hhhhh

t all group members must be present for the presentation. Should there be any issues with the schedueeheeeeerr r euue i otiuti.hhhhuiehhutheeeee teeeree uuuSmttut .teh hiuiehhihheehheeehehee rer rr uulu.rtt.uiih..iteh hhhh hheheeeer. g  eeuuuthrigritu.hhheh.ihhehhheeere et thtr ttiehtttuthiii..hhheheeeh e htheer  ttwr rertihteih.ttihhhheehhie hheeee    uteuh irtS.n  ..ttuihrehhhheiuh.eenehi rt.atr y rt...u.hi .hh.ehhnihehhhrthhree ie tr rcrhtuhniu...c.hhrtuhheherh e e ee rre rr.tttrh.i..yt.hhthh
----- diversity: 1.2
----- Generating with seed: "t all group members must be present for the presentation. Should there be any issues with the schedu"
t all group members must be present for the presentation. Should there be any issues with the schedueehhePe.e  riueieueuuit utih.htuhihhihhe.orreeerheoru frtwttritrtti,iithhhehhhhteeiheeehee e  uteer r rrr.hh..iieunhuhhiii hhhhrhhehegeei m t itte.tyirtiethwhhiihhhh uu.u.eh ee.e.r uerSrtr cbiuhuii..dhh hue,eehht.ehuhee re rr w.etr  geittahti.uh.e

oup members must be present for the presentation. Should there be any issues with the schedule, pleaere e rrr ir ttt..tt..t.huhhhhhheeeeeeeeerrrr rr t r  t....t...thihhhhhhhheeeeeee eer r   ttu..tt..r....u..uh.e.ehhieher .erre ru ttue.ttt..i...hhhhhhhhheeeeeeerr  rrrut ttt...tt..t..h.ehhhhheeeeeer r uut .rttttttu........h.h.hehheeere e rrtrrrrttt..t...t..h.thhhh.heeheeeee r rr  rrtttttttt...h.ihhhhhhhiheeeeerererr  r ttt.i..it.ih..iiuhhhhhhhheeeeeee  ere  r  tr.ttrtuth...t.h.hhhieehhh eeeee  r r
----- diversity: 1.0
----- Generating with seed: "oup members must be present for the presentation. Should there be any issues with the schedule, plea"
oup members must be present for the presentation. Should there be any issues with the schedule, pleahe  r r u . ue.hiP...hi ,.t..hhhh.hhhehreuer re e  rt ttutetu ..i.tiih.eh.hhhhheheeererur rue ttu.eudt.c..g.ttht.ihtthihhohiirhrteeeeree  refrret.tt. . .tuutt.h...l hhhteieehee erh r St  tt.rt.ntut.ut.thhtuh..uiieheeeee rr r tetrr uet r h.ifeiih.h

In [18]:
# Defining a utility function to generate new and random text based on the 
# network's learnings 
def generate_text(length, diversity): 
    # Get random starting text 
    start_index = random.randint(0, len(text) - max_length - 1) 
    generated = '' 
    sentence = text[start_index: start_index + max_length] 
    generated += sentence 
    for i in range(length): 
            x_pred = np.zeros((1, max_length, len(vocabulary))) 
            for t, char in enumerate(sentence): 
                x_pred[0, t, char_to_indices[char]] = 1.
  
            preds = model.predict(x_pred, verbose = 0)[0] 
            next_index = sample_index(preds, diversity) 
            next_char = indices_to_char[next_index] 
  
            generated += next_char 
            sentence = sentence[1:] + next_char 
    return generated 
  
print(generate_text(500, 0.2)) 

embers must be present for the presentation. Should there be any issues with the schedule, please rer   tttttt.hhhhhhhhhheeeeeeeeerr rr   utttttt...hhhhhheheeeeeerr r r   utttt.tt..hhhhhhhheeeeeeere r rr rttttttt...hhhhhhhheeeeeeeerr  rrr .utttt..th.hhhhhhhheeeeeeer rr rr ttttttiu..hhhhhhheeeeeeeeerr  r  rttttt...hhhhhhhhheeeeeeeer rr r r rtttttt.hhhhhhhhheeeeeeeee rrrr  tttt.tthihhhhhhhhheeeeeeeerrr rr r tttut.....hhhhhhheeeeeeeer r  rr rutttt....hhhhhhhheeeeeeerrr  r t tt.t......hhhhhhhheeeeeee rrrr  tttttt.t.hhhhhhhhheeeeeeee r rrrrr t tttt....hhhhhhheeeeeeee rrr r tttttt.....hhhhhheeeeeeee
